# Donnée Static Grid Model

1. Accès au dataset des lignes françaises du GRT RTE en date de 2019. Données en accès : I_max saisonniers, stations, longueurs, caractéristiques
Source : https://www.services-rte.com/en/learn-more-about-our-services/static-grid-model.html
Problèmes éventuels : données lacunaire et datées (2019)

2. Dataset : [Lignes aériennes RTE – nouveau découpage (au 10 juin 2024)](https://odre.opendatasoft.com/explore/dataset/lignes-aeriennes-rte-nv/information/?disjunctive.etat&disjunctive.tension&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6ImxvbmdpdHVkZV9kZWJ1dF9zZWdtZW50Iiwic2NpZW50aWZpY0Rpc3BsYXkiOnRydWUsImNvbG9yIjoicmFuZ2UtQWNjZW50In1dLCJ4QXhpcyI6InRlbnNpb24iLCJtYXhwb2ludHMiOm51bGwsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJldGF0Iiwic2VyaWVzQnJlYWtkb3duVGltZXNjYWxlIjoiIiwiY29uZmlnIjp7ImRhdGFzZXQiOiJsaWduZXMtYWVyaWVubmVzLXJ0ZS1udiIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuZXRhdCI6dHJ1ZSwiZGlzanVuY3RpdmUudGVuc2lvbiI6dHJ1ZX19fV0sInRpbWVzY2FsZSI6IiIsImRpc3BsYXlMZWdlbmQiOnRydWUsImFsaWduTW9udGgiOnRydWV9&location=13,45.56282,4.83351&basemap=jawg.light) / 

## Importations de librairie

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json 
import seaborn as sns
import utm

import plotly.express as px
import plotly.graph_objects as go

from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression

from utils import *

import os

## DATA Lignes

### Ligne 2019 + caractéristiques : THT

#### Ouvrir le dataset

In [ ]:
path = '/Users/matthiasmolenat/repos/congestion/data/RT_ligne_2019.csv'
df_ligne_RT_2019 = pd.read_csv(path,delimiter=';', decimal=',',index_col=False)
df_ligne_RT_2019.drop('Unnamed: 14', axis = 1, inplace=True)
df_ligne_RT_2019.head()

In [ ]:
print(f'Shapde : {df_ligne_RT_2019.shape}')
print(f'Column names : {df_ligne_RT_2019.columns}')

#### Remplacer les nan values dans les lignes s'il existe un doublon fourni en information.

In [27]:
# Group by the columns `sub_1_short`, `sub_2_short`, and `Voltage level (kV)`
group_cols = ['sub_1_short', 'sub_2_short', 'Voltage level (kV)']
df_filled = (
    df_ligne_RT_2019.groupby(group_cols)
    .apply(lambda group: group.ffill().bfill() , include_groups=False)
)
# Remove the multi-index introduced by `groupby().apply()`
df_ligne_RT_2019 = df_filled.reset_index(drop=False)

#### Drop les lignes de taille nulle

In [ ]:
df_ligne_RT_2019 =  df_ligne_RT_2019[df_ligne_RT_2019['Length (m)'] > 0]
print(f'Number of Nan values : {df_ligne_RT_2019.isna().sum().sum()}')

#### Drop les nans restants

In [29]:
df_ligne_RT_2019.dropna(inplace=True)

#### Visualisation : Lignes associées à un poste

In [ ]:
df_poste_spe = get_row_with_str(df_ligne_RT_2019, 'BAIXAS')
df_poste_spe.head(20)

#### Ajout de colonnes types

In [31]:
df_ligne_RT_2019['P_max_summer (MW)']=df_ligne_RT_2019['Voltage level (kV)']*df_ligne_RT_2019['I_max_Summer']/1000
df_ligne_RT_2019['R/length'] = df_ligne_RT_2019['Resistance R (Ω)'] / df_ligne_RT_2019['Length (m)'] 

#### Récupération de donnée types :

On cherche à obtenir des grandeurs type de caractéristiques physiques pour compléter des données lacunaires. 

Hypothèse : Pour un type de ligne donné (voltage level), les grandeurs :
- $\frac{resistance}{longueur}$,
- $\frac{reactance}{longueur}$,

se conserve. 

Une étude supplémentaire doit être mené pour la valeur de $I_{max}$

##### Résistances & réactances

In [ ]:
# Define colors for the two voltage levels
colors = {'225': 'blue', '400': 'orange'} 
X_name = 'Length (m)'
Y_name = 'Resistance R (Ω)'
scatter_plot(df_ligne_RT_2019, colors, X_name, Y_name)

In [ ]:
# Define colors for the two voltage levels
colors = {'225': 'blue', '400': 'orange'} 
X_name = 'Length (m)'
Y_name = 'Reactance X (Ω)'
scatter_plot(df_ligne_RT_2019, colors, X_name, Y_name)

In [ ]:
list_col =  ['Resistance R (Ω)', 'R/L (Ω/km)' ]
hist_plot_1(df_ligne_RT_2019, list_col)

In [ ]:
list_col =  ['Reactance X (Ω)', 'X/L (Ω/km)' ]
hist_plot_1(df_ligne_RT_2019, list_col)

##### Courant max :

In [ ]:
df_ligne_RT_2019['U/R'] = df_ligne_RT_2019['Voltage level (kV)']*1000/df_ligne_RT_2019['Resistance R (Ω)']

# Define colors for the two voltage levels
colors = {'225': 'blue', '400': 'orange'} 
X_name = 'U/R'
Y_name = 'I_max_Summer'
scatter_plot(df_ligne_RT_2019, colors, X_name, Y_name)

In [ ]:
Y_name = 'RI/U (%)' # Pertes / Puissance transitée
df_ligne_RT_2019['RI/U (%)'] = df_ligne_RT_2019['Resistance R (Ω)']*df_ligne_RT_2019['I_max_Summer']/df_ligne_RT_2019['Voltage level (kV)']/10
hist_plot_2(df_ligne_RT_2019, Y_name)

##### Résultats :

En tant que premières bonnes approximations, on prends :

In [38]:
R_per_km_225 = 0.06 # Ohm / km
X_per_km_225 = 0.402 # Ohm / km

R_per_km_400 = 0.02 # Ohm / km
X_per_km_400 = 0.302 # Ohm / km

pertes_225 = 0.355 # %
pertes_400 = 0.465 # %


##### Commentaires : 

1. Pour la modélisation des lignes, on observe dans certains cas deux types distincts et de poids équivalent. Plusieurs choix possibles : moyenne, définitions de deux modèles, etc.
2. On retrouve les chiffres R/L et X/L avancés dans https://www.techno-science.net/definition/6745.html
3. intensité max : on suppose un ratio de perte autorisé max sur la ligne. $ratio = \frac{Pertes = RI^2}{Transmis = UI}$. A Approfondir? 

### Lignes 2024 

In [18]:
path_ligne_aer = '/Users/matthiasmolenat/repos/congestion/data/lignes-aeriennes-rte-nv.csv'
df_ligne_RT_2023 = pd.read_csv(path_ligne_aer,delimiter=';')


In [ ]:
df_ligne_RT_2023.groupby('TENSION').count()

In [ ]:
df_ligne_RT_2023.shape

#### Calcul de la longueur des lignes

In [ ]:
geo_shape_index = df_ligne_RT_2023.columns.to_list().index("Geo Shape")

for index, row in df_ligne_RT_2023.iterrows():
    geo_data = json.loads(row['Geo Shape'])
    longitudes, latitudes = zip(*geo_data["coordinates"])
    df_ligne_RT_2023.at[index, "Length (m)"] = length_from_coordinate(longitudes, latitudes )
    
df_ligne_RT_2023.head()

#### Ligne avec nom de poste

In [ ]:
df_poste_spe = get_row_with_str(df_ligne_RT_2023[df_ligne_RT_2023['TENSION'] == '400kV'], 'EGUZON')
df_poste_spe.head(20)

### Extrapolation caractéristiques éléctrotechniques 2024 depuis les lignes 2019

## DATA Postes 

### DATA Postes : CAPARESEAUX

Capacté d'acceuil .csv issu de https://www.capareseau.fr/

[Données intéressantes](https://www.services-rte.com/fr/decouvrez-nos-offres-de-services/consulter-les-capacites-d-accueil-du-reseau-capareseau.html):

A l'injection :
1. Taux d'affectation des capacités  (%), part des capacités réservées du S3REnR attribuée à des installations EnR en développement ou en service. Il correspond au taux d’avancement global du schéma.

Au sous-tirage :
1. consommation minimale ( = été? en hiver, = x5 ?)

A l'injection et au sous-tirage :
1. Puissance cumulee des transformateurs existants das le poste source (MW)
2. Travaux indiqués dans le schéma ou dans son état initial

Pour la modélisation du réseau :
1. Tension amont du poste source (GRT, kV),
2. Tension aval du poste source (GRD, kV),


In [ ]:
current_file = '2501_CapacitesDAccueil'
root = '/Users/matthiasmolenat/repos/congestion/data/'
path_corrected_capa_accueil = f'{root}corrected_{current_file}.csv'

df_capareseau = pd.read_csv(path_corrected_capa_accueil,sep=',', index_col=0)
df_capareseau.head()

In [ ]:
df_capareseau = df_capareseau[df_capareseau.columns[~df_capareseau.columns.str.endswith('.1')]]

df_capareseau = df_capareseau[df_capareseau["Puissance cumulée des transformateurs existants"] > 0]
df_capareseau = df_capareseau[~df_capareseau["Consommation minimale"].isna()]
df_capareseau = df_capareseau[df_capareseau['Consommation minimale'] > 0]

df_capareseau = df_capareseau[~df_capareseau["Capacité de transformation HTB/HTA restante disponible pour l'injection sur le réseau public de distribution"].isna()]
df_capareseau = df_capareseau[df_capareseau["Capacité de transformation HTB/HTA restante disponible pour l'injection sur le réseau public de distribution"]>0]

df_capareseau.head()

In [ ]:
df_capareseau.columns

In [ ]:
df_capareseau[["Capacité réservée aux EnR au titre du S3REnR", "Capacité d'accueil réservée au titre du S3REnR, disponible vue du réseau public de transport ","Capacité de transformation HTB/HTA restante disponible pour l'injection sur le réseau public de distribution", "Puissance cumulée des transformateurs existants", "Puissance en file d'attente hors S3REnR majorée de la capacité réservée du S3REnR"]].head(20)

#### Visualisation d'une ligne en particulière

In [ ]:
df_poste_spe = get_row_with_str(df_capareseau, 'ARPEN')

for i,x in enumerate(df_poste_spe.columns.to_list()):
    print(f'\n {x} : {df_poste_spe.iloc[0][x]}')

#### Visualisation de la répartition des conso minimales 

Guess : poche de sous-tirage = p_max = p_min * 5 / p_inst, french heatmap

In [ ]:
X_name = "Puissance cumulée des transformateurs existants"
Y_name = "Consommation minimale"

df = df_capareseau[[X_name,Y_name]].copy()
df.dropna(inplace=True)

x = df[X_name]
y_min = df[Y_name]
y_max = 5*df[Y_name]

lim_acceptable = 110/100

coefficients = np.polyfit(x, y_min, 1)  # Fit a line: y = mx + c
m, c = coefficients
y_fit = m * x + c  # Regression line values

plt.figure(figsize=(12, 8))

plt.scatter(x, y_min, 
            s=20, alpha=0.3, c = 'b', label = 'Conso minimale')
plt.scatter(x, y_max,
            s=20, alpha=0.3, c = 'r', label = 'Conso max')

plt.plot(x, y_fit, label=f"Fit: y = {m:.2f}x + {c:.2f}", color="cyan")
plt.plot(x, lim_acceptable*x, label="110 % $P_{inst}$", color="black") # 110 %

# Customize the plot
plt.title(f'conso extrêmes en fonction de la puissance des transformateurs', fontsize=16)
plt.ylabel('Consommation (MW)', fontsize=12)
plt.xlabel('Puissance installée des transformateurs (MW)', fontsize=12)
plt.legend()
plt.xticks(rotation=40, ha='right')

plt.tight_layout()

# Display the plot
plt.show()

### Data Postes : ODRE

Fichier [Enceintes de poste RTE (au 10 juin 2024)](https://odre.opendatasoft.com/explore/dataset/enceintes-de-poste-rte/export/?disjunctive.etat&sort=nom_poste)

Pour, dans un premier temps, récupérer les coordonnées des postes caparéseaux ouverts précédemment

In [15]:
path_postes = '/Users/matthiasmolenat/repos/congestion/data/enceintes-de-poste-rte.csv'
df_poste_odre = pd.read_csv(path_postes,delimiter=';')

In [16]:
df_poste_odre.dropna(subset=['geo_point_2d'], inplace=True)

In [ ]:
df_poste_odre.groupby('TENSION MAXIMALE').count()

### Upgrade Data Postes : ODRE + caparéseaux

Récupération des coordonnées 2D

In [18]:
df_left =  df_capareseau.set_index('Code').copy()
df_right =  df_poste_odre.set_index('CODIFICATION NATIONALE POSTE')['geo_point_2d'].copy()

df_joined = df_left.join(df_right, how='inner')

df_joined['geo_point_2d'] = df_joined['geo_point_2d'].astype(str)
geo_point_2d = df_joined['geo_point_2d'].to_list()


df_joined['lat'] = [x.split(sep= ', ')[0] for x in geo_point_2d]
df_joined['lon'] = [x.split(sep = ', ')[-1] for x in geo_point_2d]

In [19]:
df_joined.to_csv('/Users/matthiasmolenat/repos/congestion/data/capareseau_ram.csv')

In [20]:
df_capa = df_joined.copy()

## DATA Contraintes 

1. [Contraintes résiduelles prospectives sur le RPT à horizon 3 à 5 ans (à février 2024)](https://odre.opendatasoft.com/explore/dataset/contraintes-region/export/?location=7,46.18744,3.57056&basemap=jawg.light)

In [ ]:
path = '/Users/matthiasmolenat/repos/congestion/data/contraintes-region.csv'
df_contraintes_sites = pd.read_csv(path,delimiter=';',index_col=False)
df_contraintes_sites.head()

2. [Energies et Puissances régionales liées au contraintes (vision à février 2024)](https://odre.opendatasoft.com/explore/dataset/energies-et-puissances-regionales-liees-au-contraintes/information/)

In [ ]:
path = '/Users/matthiasmolenat/repos/congestion/data/energies-et-puissances-regionales-liees-au-contraintes.csv'
df_contraintes_P = pd.read_csv(path,delimiter=';',index_col=False)
df_contraintes_P.sort_values(by='Puissance Totale à compenser', ascending= True).head(12)

## Data métropole

## Data production mineure

## Data production majeure

Moyen actuel : [fichier excel](https://www.services-rte.com/fr/telechargez-les-donnees-publiees-par-rte.html?category=generation&type=actual_generations_per_unit)

Meilleur moyen : Données RTE extraite via API Entsoe (voir code request_major_prod_entsoe.py). Pb : temps trop long. En pause.

In [35]:
path_1 =  '/Users/matthiasmolenat/repos/congestion/data/ProductionGroupe_2023-semestre1.csv'
path_2 =  '/Users/matthiasmolenat/repos/congestion/data/ProductionGroupe_2023-semestre2.csv'
path_list = [path_1, path_2]
df_list = []

for path in path_list:
    df = pd.read_csv(path, sep = ';', low_memory= False, header=1, index_col=0)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    df_list.append(df)
    
df_prod_per_plant_2023 = pd.concat(df_list)    

Récupérer concordance site / type de prod

In [36]:
df = pd.read_csv(path_1, nrows=1, sep = ';', index_col= 0)
dic_plant_type = {}

for col in df.columns:
    
    plant_name = df[col].iloc[0]
    plant_type = col.split('.')[0]
    
    dic_plant_type[plant_name] = plant_type

# Visualisation sur carte

### Visualisation Lignes

In [37]:
# Create an empty figure
fig = go.Figure()

# Param
colors_tension = {'400kV' : 'red', '225kV' : 'green',
                  '150kV' : 'blue', '90kV' : 'orange',
                  '63kV' : 'magenta', '45kV' : 'yellow',
                  '<45kV' : 'purple', 'HORS TENSION' : 'black'}
light_version = False # if true, show only starting and ending point of line
level_of_tension = ['400kV', '225kV']

if False :
    # Iterate over each row in the DataFrame
    for index, row in df_ligne_RT_2023[df_ligne_RT_2023['TENSION'].isin(level_of_tension) ].iloc[:2000].iterrows():
        # Parse the JSON string in the 'Geo Shape' column
        try:
            geo_data = json.loads(row['Geo Shape'])
        except json.JSONDecodeError:
            print(f"Error parsing JSON for row {index}. Skipping this row.")
            continue
        line_tension = row['TENSION']

        # Extract coordinates
        if "coordinates" in geo_data and isinstance(geo_data["coordinates"], list):
            longitudes, latitudes = zip(*geo_data["coordinates"])
            
            if light_version:
                longitudes = [longitudes[0], longitudes[-1]]
                latitudes = [latitudes[0], latitudes[-1]]

            # Add a trace for this line
            fig.add_trace(go.Scattermap(
                mode = "lines",
                lon = longitudes,
                lat = latitudes,
                name = f"Line {index}",  # You can replace this with a more meaningful name if available
                line = dict(width = 2),
                marker_color = colors_tension[line_tension]
            ))
        else:
            print(f"No valid coordinates found for row {index}. Skipping this row.")

    # Update the layout
    fig.update_layout(
        geo=dict(
            scope='europe',  # You can change this to a specific country or region
            projection_type='natural earth',
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
            coastlinecolor='rgb(204, 204, 204)',
            showocean=True,
            oceancolor='rgb(230, 230, 250)',
            center=dict(
                lon=longitudes[0],  # Center on the first longitude
                lat=latitudes[0]    # Center on the first latitude
            ),
        ),
        showlegend=False,
        title='Some lines in France'
        )
    # Show the figure
    fig.show()



### Visualisation postes capareseau

#### sous-tirage

In [38]:
fig_title = 'RAM_soutirage during hivernal consumption - CAPARESEAU datas'
values_title = 'RAM_soutirage'

df_capa['text'] = df_capa['Nom'] + ', Value = ' + df_capa[values_title].astype(str) + 'MW , P_inst = ' + df_capa['Puissance cumulée des transformateurs existants'].astype(str) + ' MW'

# show_map_substation_gradient(df_capa, values_title,fig_title,values_title, 'text', True)

#### Injections

In [39]:
fig_title = 'RAM_injection during summer - CAPARESEAU datas'
values_title = "RAM_injection"

df_capa['text'] = df_capa['Nom'].astype(str)  + ', value = ' + df_capa[values_title].astype(str) + ' MW'
bar_title = values_title

# show_map_substation_gradient(df_capa,values_title,fig_title,bar_title, scale_reverse= True)

#### intersections

In [ ]:
fig_title = 'Substation RAM : min injection & consumption - CAPARESEAU datas'

values_title = "min_RAM"
df_capa['text'] = df_capa['Nom'].astype(str)  + ', value = ' + df_capa[values_title].astype(str) + ' MW, P_inst = ' + df_capa['Puissance cumulée des transformateurs existants'].astype(str) + ' MW'
bar_title = values_title  + ' MW'

show_map_substation_gradient(df_capa,values_title,fig_title,bar_title, scale_reverse= True)

#### Navigation dans les datasets  et comparaison avec D1

In [ ]:
path_D1 = '/Users/matthiasmolenat/repos/congestion/data/00.Suivi D1  - Suivi_D1.csv'
df_D1 = pd.read_csv(path_D1, header=1, index_col=0)
df_D1.head()

In [42]:
# with open('compared_D1_ratio.txt', 'a') as file:
    
#     for index, row in df_D1.iterrows():
#         df_poste_spe = get_row_with_str(df_intersection, str(index))

#         if not df_poste_spe.empty and not pd.isna(row['Validation comité']) :
#             file.write(f"{index}, demande : 50 MW?, results :  {row['Validation comité']}" + '\n') 
#             file.write(f"injection RAM = {df_poste_spe['RAM_injection'].iloc[0]} MW, vs limitations perennes = {row['Injection']} h, temporaires = {row['Injection.1']} h" + '\n') 
#             file.write(f"sous-tirage RAM = {df_poste_spe['RAM_soutirage'].iloc[0]} MW, vs limitations perenne = {row['Soutirage']} h, temporaire = {row['Soutirage.1']} h" + '\n') 
#             file.write('\n')
#             file.write('\n')
            
        

#### Navigation dans les datasets : obtenir les postes CAPARESEAU les plus proches

In [41]:
my_GeopointPoste_1 = [48.5587213, 7.7976421]
my_GeopointPoste_2 = [49.3132494, 1.1897809]
my_GeopointPoste_3 = [49.263092, 4.100068] # Cernay
my_GeopointPoste_4 = [46.754655, 5.915800] # Champagnolle
my_GeopointPoste_5 = [48.7098768941521, 1.3839132342552853] # Les Arpents
my_GeopointPoste_6 = [45.725201, 5.699428] # Brens 
my_GeopointPoste_7 = [42.542426, 3.028624] # Argelès sur mer
my_GeopointPoste_8 = [42.752815, 2.801412] # Baixas
n_voisins = 20

df_res = get_closest_post(df_capa,my_GeopointPoste_8, n_voisins)

In [ ]:
df_res = df_res[["Nom", "dist_to_my_poste_km","RAM_injection", "RAM_soutirage", "Capacité de transformation HTB/HTA restante disponible pour l'injection sur le réseau public de distribution"]]
df_res.head(20)
